In [1]:
import os
import numpy as np
import cv2 as cv
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import h5py

from models.resnet_custom import resnet50_baseline
from utils.utils import print_network, collate_features

In [2]:
patch_dir = "image_sets/patches/"
feat_dir = "image_sets/features/"
actual_feat_dir = "image_sets/patches/fungal_vs_nonfungal_resnet_features/pt_files/"

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Create feat_dir if not exists. 
# Not properly fixed
if not os.path.exists(feat_dir):
    try:
        print("Features directory doesn't exist. Creating ...")
        os.mkdir(feat_dir, exist_ok=True)
    except:
        print("ERROR: Cannot create the Features directory")

model = resnet50_baseline(pretrained=True)
model = model.to(device)

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model.eval()

ResNet_Baseline(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck_Baseline(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(

In [4]:
# patch_folders = [os.path.join(patch_dir, folder) for folder in sorted(os.listdir(patch_dir))]
# patches_per_image = len(os.listdir(patch_folders[0]))
# print(patches_per_image)

In [5]:
# Create dataset from the image patches
for folder in sorted(os.listdir(patch_dir)):
    patch_folder = os.path.join(patch_dir, folder)
    dataset = []
    for patch_file in sorted(os.listdir(patch_folder)):
        if patch_file == "pt_files":
            continue
        
        img_path = os.path.join(patch_folder, patch_file)

        img = Image.open(img_path)

        img_arr = np.asarray(img)
        # img_arr = np.expand_dims(img_arr, 0)
        # img_PIL = Image.fromarray(img_arr)

        # Create the dataset loader
        imgs = torch.tensor(img_arr)

        # Get coord in [x, y] format
        coord = img_path.split("/")
        coord = coord[-1]
        coord = coord.split(".")[-2]
        coord = coord.split("_")
        coord = [int(coord[-2])/256, int(coord[-1])/256]

        dataset.append([imgs, coord])
    
    loader = DataLoader(dataset=dataset, batch_size=1)    
    filename = str(folder).split("/")[-1]
    print("File:", filename)
    
    features = []
    for count, data in enumerate(loader):
        with torch.no_grad():
            coord = data[1]
            batch = data[0]
            batch = torch.unsqueeze(batch, 0)
            batch = batch.reshape([1, 3, 256, 256])
            batch = batch.to(device, non_blocking=True)
            batch = batch.float()

            feature = model(batch)
            feature = feature.cpu().numpy()
            feature = torch.from_numpy(feature)
            feature = np.expand_dims(feature, 0)
            
            # Group the features
            features.append(feature)
    
    # To Tensor    
    features = np.asarray(features, dtype="float32")   
    features = torch.tensor(features)

    print(filename)
    filePath = os.path.join(feat_dir, filename+'.pt')
    print(count, " || ", coord, " || ", features, " || ", filePath)
    # print("Features size: ", features.shape)
    torch.save(features, filePath)

    # Save the .hdf5
    # hf = h5py.File('data.h5', 'w')

    print("="*15)

File: F005a02
F005a02
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.4815,  0.0000,  2.4632,  ..., 30.7341,  0.0000,  0.0000]]],


        [[[ 0.4938,  0.0000,  2.5673,  ..., 30.4145,  0.0000,  0.0000]]],


        [[[ 0.5683,  0.0000,  2.3696,  ..., 30.8572,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4234,  0.0000,  1.8237,  ..., 31.4962,  0.0000,  0.0000]]],


        [[[ 0.4894,  0.0000,  2.6134,  ..., 30.8446,  0.0000,  0.0000]]],


        [[[ 0.5468,  0.0000,  1.9466,  ..., 32.8999,  0.0000,  0.0000]]]])  ||  image_sets/features/F005a02.pt
File: F006a01
F006a01
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.3182,  0.0000,  0.8749,  ..., 26.4158,  0.0000,  0.0000]]],


        [[[ 0.3066,  0.0000,  0.8308,  ..., 26.3428,  0.0000,  0.0000]]],


        [[[ 0.3007,  0.0000,  0.7373,  ..., 25.6035,  0.0000,  0.0000]]],


        ...,


        [[[ 0.3405,  0.

F007a04
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.4846,  0.0000,  1.3858,  ..., 31.0430,  0.0000,  0.0000]]],


        [[[ 0.4770,  0.0000,  1.2068,  ..., 31.2870,  0.0000,  0.0000]]],


        [[[ 0.5107,  0.0000,  1.7356,  ..., 28.2466,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4660,  0.0000,  1.7475,  ..., 29.4997,  0.0000,  0.0000]]],


        [[[ 0.4612,  0.0000,  1.7134,  ..., 27.3658,  0.0000,  0.0000]]],


        [[[ 0.4547,  0.0000,  1.3337,  ..., 25.1211,  0.0000,  0.0000]]]])  ||  image_sets/features/F007a04.pt
File: F007a05
F007a05
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.4957,  0.0000,  1.3979,  ..., 30.6404,  0.0000,  0.0000]]],


        [[[ 0.4867,  0.0000,  1.2333,  ..., 30.8226,  0.0000,  0.0000]]],


        [[[ 0.5132,  0.0000,  1.7169,  ..., 27.8564,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4665,  0.0000,  1.5788,

F007a19
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.3606,  0.0000,  1.1517,  ..., 30.7981,  0.0000,  0.0000]]],


        [[[ 0.3703,  0.0000,  1.0613,  ..., 30.9283,  0.0000,  0.0000]]],


        [[[ 0.4452,  0.0000,  3.9496,  ..., 24.4262,  0.0000,  0.0000]]],


        ...,


        [[[ 0.7555,  0.0000,  2.6720,  ..., 21.7873,  0.0000,  0.0000]]],


        [[[ 0.4985,  0.0000,  2.4050,  ..., 23.3936,  0.0000,  0.0000]]],


        [[[ 0.9199,  0.0000,  9.4447,  ..., 17.6042,  0.0000,  0.0000]]]])  ||  image_sets/features/F007a19.pt
File: F007a21
F007a21
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.4110,  0.0000,  1.1812,  ..., 23.8818,  0.0000,  0.0000]]],


        [[[ 0.3735,  0.0000,  1.3613,  ..., 23.5286,  0.0000,  0.0000]]],


        [[[ 0.4105,  0.0000,  1.1318,  ..., 23.3653,  0.0000,  0.0000]]],


        ...,


        [[[ 0.3750,  0.0000,  1.1361,

F012a01
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.3570,  0.0000,  1.0344,  ..., 32.5288,  0.0000,  0.0000]]],


        [[[ 0.3574,  0.0000,  1.0284,  ..., 32.6087,  0.0000,  0.0000]]],


        [[[ 0.3582,  0.0000,  1.0430,  ..., 32.5181,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4474,  0.0000,  1.5380,  ..., 28.5980,  0.0000,  0.0000]]],


        [[[ 0.3550,  0.0000,  0.9648,  ..., 31.5814,  0.0000,  0.0000]]],


        [[[ 0.4972,  0.0000,  2.2788,  ..., 28.8051,  0.0000,  0.0000]]]])  ||  image_sets/features/F012a01.pt
File: F012a02
F012a02
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.4038,  0.0000,  1.2536,  ..., 36.8186,  0.0000,  0.0000]]],


        [[[ 0.4171,  0.0000,  1.1132,  ..., 37.2017,  0.0000,  0.0000]]],


        [[[ 0.4506,  0.0000,  1.2165,  ..., 36.8925,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4477,  0.0000,  3.5505,

F013a10
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.5108,  0.0000,  1.7617,  ..., 32.4205,  0.0000,  0.0000]]],


        [[[ 0.4467,  0.0000,  2.0369,  ..., 31.9419,  0.0000,  0.0000]]],


        [[[ 0.4412,  0.0000,  1.8714,  ..., 30.9281,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4412,  0.0000,  2.1926,  ..., 29.2496,  0.0000,  0.0000]]],


        [[[ 0.3963,  0.0000,  1.4071,  ..., 31.7057,  0.0000,  0.0000]]],


        [[[ 0.4474,  0.0000,  1.5065,  ..., 31.8727,  0.0000,  0.0000]]]])  ||  image_sets/features/F013a10.pt
File: F013a11
F013a11
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.3997,  0.0000,  1.2427,  ..., 35.9953,  0.0000,  0.0000]]],


        [[[ 1.1237,  0.0000,  6.0386,  ..., 29.9611,  0.0000,  0.0000]]],


        [[[ 0.7090,  0.0000,  2.9673,  ..., 30.7303,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4495,  0.0000,  1.5299,

...

N004a108
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.3761,  0.0000,  1.8167,  ..., 28.7333,  0.0000,  0.0000]]],


        [[[ 0.3709,  0.0000,  1.8264,  ..., 28.4209,  0.0000,  0.0000]]],


        [[[ 0.3599,  0.0000,  1.4904,  ..., 28.3546,  0.0000,  0.0000]]],


        ...,


        [[[ 0.3935,  0.0000,  2.0692,  ..., 27.9469,  0.0000,  0.0000]]],


        [[[ 0.3967,  0.0000,  1.6863,  ..., 27.8162,  0.0000,  0.0000]]],


        [[[ 0.4448,  0.0000,  2.0068,  ..., 25.5159,  0.0000,  0.0000]]]])  ||  image_sets/features/N004a108.pt
File: N004a109
N004a109
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.3946,  0.0000,  1.5996,  ..., 29.6484,  0.0000,  0.0000]]],


        [[[ 0.4053,  0.0000,  2.0567,  ..., 28.7006,  0.0000,  0.0000]]],


        [[[ 0.4877,  0.0000,  2.7191,  ..., 27.2735,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4777,  0.0000,  2.5

N004a122
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.5767,  0.0000,  1.7566,  ..., 51.7659,  0.0000,  0.0000]]],


        [[[ 0.5633,  0.0000,  2.4713,  ..., 45.3688,  0.0000,  0.0000]]],


        [[[ 0.5460,  0.0000,  5.4090,  ..., 41.2890,  0.0000,  0.0000]]],


        ...,


        [[[ 0.5287,  0.0000,  3.5066,  ..., 30.6000,  0.0000,  0.0000]]],


        [[[ 0.4707,  0.0000,  2.3082,  ..., 33.1786,  0.0000,  0.0000]]],


        [[[ 0.4620,  0.0000,  2.3855,  ..., 33.5062,  0.0000,  0.0000]]]])  ||  image_sets/features/N004a122.pt
File: N004a123
N004a123
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.5232,  0.0000, 10.5559,  ..., 18.8467,  0.0000,  0.0000]]],


        [[[ 0.3858,  0.0000,  2.1888,  ..., 28.6519,  0.0000,  0.0000]]],


        [[[ 0.4548,  0.0000,  2.7400,  ..., 28.8614,  0.0000,  0.0000]]],


        ...,


        [[[ 0.3733,  0.0000,  2.9

N004a136
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.5744,  0.0000,  1.8474,  ..., 51.8672,  0.0000,  0.0000]]],


        [[[ 0.7940,  0.0000,  2.6575,  ..., 46.8654,  0.0000,  0.0000]]],


        [[[ 0.5327,  0.0000,  2.4123,  ..., 39.6473,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4562,  0.0000,  1.7984,  ..., 35.9333,  0.0000,  0.0000]]],


        [[[ 0.5038,  0.0000,  2.0901,  ..., 34.4356,  0.0000,  0.0000]]],


        [[[ 0.4623,  0.0000,  2.1090,  ..., 33.8170,  0.0000,  0.0000]]]])  ||  image_sets/features/N004a136.pt
File: N004a137
N004a137
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.6113,  0.0000,  1.7145,  ..., 51.9952,  0.0000,  0.0000]]],


        [[[ 0.5682,  0.0000,  1.8127,  ..., 52.2431,  0.0000,  0.0000]]],


        [[[ 0.6069,  0.0000,  2.0678,  ..., 51.2212,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4773,  0.0000,  2.5

N005a013
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.7982,  0.0000,  5.7367,  ..., 26.8819,  0.0000,  0.0000]]],


        [[[ 0.4960,  0.0000,  5.4719,  ..., 31.6351,  0.0000,  0.0000]]],


        [[[ 0.5078,  0.0000,  2.8825,  ..., 30.9427,  0.0000,  0.0000]]],


        ...,


        [[[ 0.8070,  0.0000,  3.5759,  ..., 29.5956,  0.0000,  0.0000]]],


        [[[ 0.8365,  0.0000, 19.3595,  ..., 22.6028,  0.0000,  0.0000]]],


        [[[ 0.8735,  0.0000, 10.4839,  ..., 34.2293,  0.0000,  0.0000]]]])  ||  image_sets/features/N005a013.pt
File: N005a014
N005a014
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.5211,  0.0000,  2.3725,  ..., 43.8926,  0.0000,  0.0000]]],


        [[[ 0.6748,  0.0000,  6.1981,  ..., 35.8152,  0.0000,  0.0000]]],


        [[[ 0.5108,  0.0000,  1.3291,  ..., 46.5516,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4682,  0.0000,  2.9

N006a005
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.4867,  0.0000,  3.4270,  ..., 21.9078,  0.0000,  0.0000]]],


        [[[ 0.4886,  0.0000,  6.5097,  ..., 18.1999,  0.0000,  0.0000]]],


        [[[ 0.5608,  0.0000,  3.8894,  ..., 15.1060,  0.0000,  0.0000]]],


        ...,


        [[[ 0.8215,  0.0000,  3.7795,  ..., 18.2124,  0.0000,  0.0000]]],


        [[[ 0.7042,  0.0000,  4.6563,  ..., 19.2272,  0.0000,  0.0000]]],


        [[[ 0.6671,  0.0000,  3.7720,  ..., 25.8811,  0.0000,  0.0000]]]])  ||  image_sets/features/N006a005.pt
File: N006a006
N006a006
23  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[[[ 0.5335,  0.0000,  4.7606,  ..., 14.0997,  0.0000,  0.0000]]],


        [[[ 0.4944,  0.0000,  3.8565,  ..., 16.5534,  0.0000,  0.0000]]],


        [[[ 0.6210,  0.0000,  3.4421,  ..., 23.6889,  0.0000,  0.0000]]],


        ...,


        [[[ 0.4699,  0.0000,  1.9

In [7]:
actual_feat_dir = actual_feat_dir
feat_dir = feat_dir
!mkdir -p $actual_feat_dir
!mv $feat_dir/* $actual_feat_dir

!rm -rf "${actual_feat_dir}fungal_vs_nonfungal_resnet_features.pt"

mv: cannot stat 'image_sets/features//*': No such file or directory
